this script will build and deploy a model to perform crop type classification using random forest

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=MmTW7G2bnc8DrSbtFFG_vGRLEGFRCAUSCnf3NznlWxU&tc=jraMmBN_A8aIhSMRFdp2yiTAGEUFbXm0gD7xE1ORCrw&cc=Nb_sYeuf7gP6PcvknDGvwwCOfm9W3vEUkjxCmzYT60k

The authorization workflow will generate a code, which you should paste in the box below. 


KeyboardInterrupt: ignored

In [ ]:
# import packages
!pip install geemap
import geemap
import pandas as pd
import numpy as np
!pip install geopandas
import geopandas as gpd
%matplotlib inline 
import matplotlib.pyplot as plt

In [ ]:
# initialize geemap
geemap.ee_initialize()

In [ ]:
# define mapvariable
Map = geemap.Map()

In [ ]:
# import the data
in_csv1 = '/content/drive/MyDrive/MIZANI FIELDWORK DATA/githunguri_labeled_data.csv'


In [ ]:
# clean the data to remain with coordinates and crop type classification
# remove null values
df1 = geemap.csv_to_pandas(in_csv1)
df1 = df1.dropna()
df1.describe()

In [ ]:
# sample first 5 features
df1.head()

,X,Y,classification,description
0,36.687597,-1.072014,206,Barren Land
1,36.688171,-1.071127,206,Barren Land
2,36.689757,-1.070010,206,Barren Land
3,36.684268,-1.071294,206,Barren Land
4,36.690655,-1.075949,206,Barren Land


In [ ]:
# convert to feature collection

features=[]
# Define feature with a geometry and 'name' field from the dataframe
for index, row in df1.iterrows():
    geom=ee.Geometry.Point([row['X'],row['Y']])
    feature = ee.Feature(geom).set({'classification': row['classification']})
    features.append(feature)
fc1 = ee.FeatureCollection(features)

#fc1.getInfo()


In [ ]:
Map.setCenter(36.777911, -1.058634, 14)

#define bbox 
bBox = ee.Geometry.BBox(36.670697, -1.041734, 36.717424, -1.087474)

# Function to mask clouds based on the pixel_qa band of Landsat SR data.

def cloudMaskL8(image):
  qa = image.select('QA_PIXEL')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
  cloud = qa.bitwiseAnd(1 << 4) \
                  .And(qa.bitwiseAnd(1 << 3)) \
                  .Or(qa.bitwiseAnd(1 << 2))
  # Remove edge pixels that don't occur in all bands
  mask2 = image.mask().reduce(ee.Reducer.min())
  return image.updateMask(cloud.Not()).updateMask(mask2)

# Applies scaling factors.
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)

# load processed landsat image

dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2022-03-01', '2022-03-31') \
    .map(cloudMaskL8) \
    .map(applyScaleFactors) \
    .median()

#dataset = dataset.median()

visualization = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0.0,
  'max': 0.3,
}

Map.addLayer(dataset.clip(bBox), visualization)

# calculate indices and add to map
def addIndices(image):
  ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename(['ndvi'])
  ndbi = image.normalizedDifference(['SR_B6', 'SR_B6']).rename(['ndbi'])
  ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename(['ndwi'])
  TCB = dataset.expression(
    '0.3029 * SR_B2 + 0.2786 * SR_B3 + 0.4733 * SR_B4 + 0.5599 * SR_B5 + 0.508 * SR_B6 + 0.1872 * SR_B7', {
      'SR_B2': image.select('SR_B2'),
      'SR_B3': image.select('SR_B3'),
      'SR_B4': image.select('SR_B4'),
      'SR_B5': image.select('SR_B5'),
      'SR_B6': image.select('SR_B6'),
      'SR_B7': image.select('SR_B7')
    }).rename('TCB')
  bsi = image.expression(
      '(( X + Y ) - (A + B)) /(( X + Y ) + (A + B)) ', {
        'X': image.select('SR_B4'), #red
        'Y': image.select('SR_B6'),  #swi1
        'A': image.select('SR_B5'), # nir
        'B': image.select('SR_B2'), # blue
  }).rename('bsi')
  return image.addBands(ndvi).addBands(ndbi).addBands(ndwi).addBands(bsi)

dataset = addIndices(dataset)

# Function to Normalize Image
def normalize(image):
  bandNames = image.bandNames()
  # Compute min and max of the image
  minDict = image.reduceRegion(**{
    'reducer': ee.Reducer.min(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9,
    'bestEffort': True,
    'tileScale': 16
  })
  maxDict = image.reduceRegion(**{
    'reducer': ee.Reducer.max(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9,
    'bestEffort': True,
    'tileScale': 16
  })
  mins = ee.Image.constant(minDict.values(bandNames))
  maxs = ee.Image.constant(maxDict.values(bandNames))

  normalized = image.subtract(mins).divide(maxs.subtract(mins))
  return normalized

# Function to Standardize Image

def standardize(image):
  bandNames = image.bandNames()
  # Mean center the data to enable a faster covariance reducer
  # and an SD stretch of the principal components.
  meanDict = image.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9,
    'bestEffort': True,
    'tileScale': 16
  })
  means = ee.Image.constant(meanDict.values(bandNames))
  centered = image.subtract(means)

  stdDevDict = image.reduceRegion(**{
    'reducer': ee.Reducer.stdDev(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9,
    'bestEffort': True,
    'tileScale': 16
  })
  stddevs = ee.Image.constant(stdDevDict.values(bandNames))

  standardized = centered.divide(stddevs)

  return standardized

dataset= normalize(dataset)
dataset = standardize(dataset)

#define training image
trainImage = dataset.sampleRegions(**{
    'collection': fc1,
    'properties':["classification"],
    'scale':30,
    'geometries': True
})

#split the data
trainData = trainImage.randomColumn()
split_thresh = 0.7

train = trainData.filter(ee.Filter.lt('random', split_thresh))
test = trainData.filter(ee.Filter.gte('random',split_thresh))

# create random forest classifier 

# Train a CART classifier (up to 30 leaf nodes in each tree) from the
# training sample.
classifier = ee.Classifier.smileCart(200,2)

# fit the classifer
bands = ['SR_B1',
   'SR_B2',
   'SR_B3',
   'SR_B4',
   'SR_B5',
   'SR_B6',
   'SR_B7',
   'ndvi',
   'ndbi',
   'ndwi',
   'bsi']
trainedClassifier = classifier.train(train, "classification", bands)

# Classify the image.
classified = dataset.classify(trainedClassifier)

#display the map
#visualize classification
Map.addLayer(classified.clip(bBox),
             {'min': 2, 'max': 206, 
              'palette': [
                          '#5F9EA0',	
                          '#7FFF00',	
                          '#D2691E',	
                          '#FF7F50',	
                          '#6495ED',	
                          '#FFF8DC',	
                          '#DC143C',	
                          '#00FFFF',	
                          '#00008B',	
                          '#008B8B',	
                          '#B8860B',	
                          '#A9A9A9',	
                          '#006400',	
                          '#BDB76B',	
                          '#8B008B',
                          '#556B2F',	
                          '#FF8C00',	
                          '#9932CC',	
                          '#8B0000',	
                          '#E9967A',	
                          '#8FBC8F',	
                          '#483D8B',	
                          '#2F4F4F',
                          '#FF1493',
                          '#00FF00',
                          '#A0522D']},
             'classified')
	

Map.setCenter(36.6940605, -1.064604, 14)
Map


Map(center=[-1.064604, 36.6940605], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [ ]:
# Get a confusion matrix and overall accuracy for the training sample.
trainAccuracy = trainedClassifier.confusionMatrix()

print('train accuracy ',trainAccuracy.accuracy().getInfo())
print('train kappa ',trainAccuracy.kappa().getInfo())

#classify test set
testClassifier = classifier.train(test, "classification", bands)

# Get a confusion matrix and overall accuracy for the training sample.
testAccuracy = testClassifier.confusionMatrix()

print('test accuracy ',testAccuracy.accuracy().getInfo())
print('test kappa ', testAccuracy.kappa().getInfo())

train accuracy  0.8080808080808081
train kappa  0.7781525687597302
test accuracy  0.8018433179723502
test kappa  0.7701441064170463


In [ ]:
# The Sentinel Dataset
S2= ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

def maskS2clouds(image):
  qa = image.select('QA60')


  # Bits 10 and 11 are clouds and cirrus, respectively
  cloudBitMask= 1<<10
  cirrusBitMask = 1<<11
  #setting both flags to zero to indicate clear conditions
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

#Maping the fucntion over an epoch

image_S2 = ee.ImageCollection("COPERNICUS/S2").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                      .filterDate('2022-01-01', '2022-03-31') \
                      .map(maskS2clouds) \
                      .median()
rgbVis= {
   'min': 0.004,
   'max':0.39,
   'bands': ['B11', 'B8', 'B4']
 }


# calculate indices and add to map
def addIndices(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
  ndbi = image.normalizedDifference(['B6', 'B6']).rename(['ndbi'])
  ndwi = image.normalizedDifference(['B8', 'B12']).rename(['ndwi'])
  bsi = image.expression(
      '(( X + Y ) - (A + B)) /(( X + Y ) + (A + B)) ', {
        'X': image.select('B11'), #red
        'Y': image.select('B4'),  #swi1
        'A': image.select('B8'), # nir
        'B': image.select('B2'), # blue
  }).rename('bsi')
  return image.addBands(ndvi).addBands(ndbi).addBands(ndwi).addBands(bsi)

image_S2 = addIndices(image_S2)

Map.setCenter(36.6940605, -1.064604, 14)
#define bbox 
bBox = ee.Geometry.BBox(36.670697, -1.041734, 36.717424, -1.087474)
Map.addLayer(image_S2.clip(bBox))

#define training image on sentinel 2
trainImgSent2 = image_S2.sampleRegions(**{
    'collection': fc1,
    'properties':["classification"],
    'scale':30,
    'geometries': True
})

#split the data
trainImgSent2 = trainImgSent2.randomColumn()
split_thresh = 0.7

trainSent2 = trainImgSent2.filter(ee.Filter.lt('random', split_thresh))
testSent2 = trainImgSent2.filter(ee.Filter.gte('random',split_thresh))

trainSent2.getInfo()

# Train a CART classifier (up to 30 leaf nodes in each tree) from the
# training sample.
classifierSent2 = ee.Classifier.smileCart(200,2)

# fit the classifer
bandsSent2 = ['B1',
   'B2',
   'B3',
   'B4',
   'B5',
   'B6',
   'B7',
   'B8',
   'B8A',
   'B9',
   'B10',
   'B11',
   'B12',
   'ndvi',
   'ndbi',
   'ndwi',
   'bsi']
trainedClassifierSent2 = classifierSent2.train(trainSent2, "classification", bandsSent2)

# Classify the image.
classifiedSent2 = image_S2.classify(trainedClassifierSent2)


#display the map
#visualize classification
Map.addLayer(classifiedSent2.clip(bBox),
             {'min': 2, 'max': 206, 
              'palette': [
                          '#5F9EA0',	
                          '#7FFF00',	
                          '#D2691E',	
                          '#FF7F50',	
                          '#6495ED',	
                          '#FFF8DC',	
                          '#DC143C',	
                          '#00FFFF',	
                          '#00008B',	
                          '#008B8B',	
                          '#B8860B',	
                          '#A9A9A9',	
                          '#006400',	
                          '#BDB76B',	
                          '#8B008B',
                          '#556B2F',	
                          '#FF8C00',	
                          '#9932CC',	
                          '#8B0000',	
                          '#E9967A',	
                          '#8FBC8F',	
                          '#483D8B',	
                          '#2F4F4F',
                          '#FF1493',
                          '#00FF00',
                          '#A0522D']},
             'classified')
	

Map.setCenter(36.6940605, -1.064604, 14)
Map

Map(bottom=2109856.0, center=[-1.064604, 36.6940605], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Get a confusion matrix and overall accuracy for the training sample.
trainAccuracySent2 = trainedClassifierSent2.confusionMatrix()
print('Training accuracy', trainAccuracySent2.accuracy().getInfo())
print('Training kappa ', trainAccuracySent2.kappa().getInfo())

#classify test set
testClassifierSent2 = classifierSent2.train(testSent2, "classification", bandsSent2)

# Get a confusion matrix and overall accuracy for the training sample.
testAccuracySent2 = testClassifierSent2.confusionMatrix()

print('Test accuracy', testAccuracySent2.accuracy().getInfo())
print('Test kappa',testAccuracySent2.kappa().getInfo())

Training accuracy 0.8228346456692913
Training kappa  0.7937381575385726
Test accuracy 0.7990196078431373
Test kappa 0.7710625718508787


In [ ]:
areaImage = ee.Image.pixelArea().divide(1e6).addBands(classified)

# Calculate Area by Class
# Using a Grouped Reducer
areas = areaImage.reduceRegion(**{
      'reducer': ee.Reducer.sum().group(**{
      'groupField': 1,
      'groupName': 'classification',
    }),
    'geometry': bBox,
    'scale': 30
    })

classAreas = ee.List(areas.get('groups'))
print(classAreas)

# export as csv
mytask = ee.batch.Export.table.toDrive(collection=classAreas, 
                                       folder='/content/drive/MyDrive/ content drive MyDrive', 
                                       description='crop_areas_rf', 
                                       fileFormat='CSV')

In [ ]:
# define legend dictionary
legend_dict = {
    2: 'cadetblue',                  # Avocado         
    6: 'chartreuse',             # Bay Leaves       
    13: 'chocolate',               #  Calliandra      
    24: 'coral',                    #  Coriander      
    17: 'cornflowerblue',         # Cauliflower       
    3: 'cornsilk',                   #Bananas         
    27: 'crimson',                  #Courgette        
    33: 'cyan',     #Exotic Chinese Vegetables        
    10: 'darkblue',                  #Broccoli        
    103: 'darkcyan',                   #Spinach      
    115: 'darkgoldenrod',	       #Thorned Melon      
    25: 'darkgray',	               #Corn Maize       
    76: 'darkgreen',	           #Passion Fruit       
    12: 'darkkhaki',                  #Cabbage       
    89: 'darkmagenta',                #Pumpkin       
    70: 'darkolivegreen',	             #Onions       
    118: 'darkorange',             #Tree Tomato      
    93: 'darkorchid',              #Red Radish       
    87: 'darkred',                     #Potato      
    108: 'darksalmon',             #Sukuma Wiki     
    57: 'darkseagreen',                 #Maize      
    67: 'darkslateblue',         #Napier Grass      
    113:'darkslategray',                    #Tea 
}
	
Map.add_legend(legend_dict=legend_dict)
Map

Map(bottom=2109856.0, center=[-1.064604, 36.6940605], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
classified.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 2147483647,
    'min': -2147483648,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'classification'}],
 'type': 'Image'}

In [ ]:
dataset.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 13.595968790787326,
    'min': -4.899155976158784,
    'precision': 'double',
    'type': 'PixelType'},
   'id': 'SR_B1'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 13.709589041324502,
    'min': -4.996635899478313,
    'precision': 'double',
    'type': 'PixelType'},
   'id': 'SR_B2'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 14.982056153795222,
    'min': -5.625856198588049,
    'precision': 'double',
    'type': 'PixelType'},
   'id': 'SR_B3'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 14.988747133610564,
    'min': -5.543953261893737,
    'precision': 'double',
    'type': 'PixelType'},
   'id': 'SR_B4'},
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 17.475650783548808,
    'min': -10.724361694846278,
    'precisi

In [ ]:
# ndvi properties in dataset
'''
  {'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0],
   'data_type': {'max': 4.287969859172442,
    'min': -7.6952461046816225,
    'precision': 'double',
    'type': 'PixelType'},
   'id': 'ndvi'},
'''
NDVI = dataset.select('ndvi')
Map.addLayer(NDVI.clip(bBox), vis_params = {'min': -7.6952461046816225, 
                                            'max': 4.287969859172442,
                                            'palette':[
                                                       'darkred',
                                                       'red',
                                                       'yellow',
                                                       'limegreen',
                                                       'darkgreen'
                                            ]})
Map

Map(bottom=2109865.0, center=[-1.064582599571565, 36.69407844543458], controls=(WidgetControl(options=['positi…

In [ ]:
# The Sentinel Dataset
S2= ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

def maskS2clouds(image):
  qa = image.select('QA60')


  # Bits 10 and 11 are clouds and cirrus, respectively
  cloudBitMask= 1<<10
  cirrusBitMask = 1<<11
  #setting both flags to zero to indicate clear conditions
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

#Maping the fucntion over an epoch

image_S2 = ee.ImageCollection("COPERNICUS/S2").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                      .filterDate('2022-01-01', '2022-03-31') \
                      .map(maskS2clouds) \
                      .median()
rgbVis= {
   'min': 0.004,
   'max':0.39,
   'bands': ['B11', 'B8', 'B4']
 }

In [ ]:
baringo = ee.Geometry.BBox(35.528285, 0.170590, 35.840163, -0.097081)

# Function to mask clouds based on the pixel_qa band of Landsat SR data.

def cloudMaskL8(image):
  qa = image.select('QA_PIXEL')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
  cloud = qa.bitwiseAnd(1 << 4) \
                  .And(qa.bitwiseAnd(1 << 3)) \
                  .Or(qa.bitwiseAnd(1 << 2))
  # Remove edge pixels that don't occur in all bands
  mask2 = image.mask().reduce(ee.Reducer.min())
  return image.updateMask(cloud.Not()).updateMask(mask2)

# Applies scaling factors.
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)

# load processed landsat image

june = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2021-06-01', '2022-06-30') \
    .map(cloudMaskL8) \
    .map(applyScaleFactors) \
    .median()

visualization = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 0.0,
  'max': 0.3,
}

Map.addLayer(june.clip(baringo), visualization, 'june')

# calculate indices and add to map
def addIndices(image):
  ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename(['ndvi'])
  ndbi = image.normalizedDifference(['SR_B6', 'SR_B6']).rename(['ndbi'])
  ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename(['ndwi'])
  TCB = dataset.expression(
    '0.3029 * SR_B2 + 0.2786 * SR_B3 + 0.4733 * SR_B4 + 0.5599 * SR_B5 + 0.508 * SR_B6 + 0.1872 * SR_B7', {
      'SR_B2': image.select('SR_B2'),
      'SR_B3': image.select('SR_B3'),
      'SR_B4': image.select('SR_B4'),
      'SR_B5': image.select('SR_B5'),
      'SR_B6': image.select('SR_B6'),
      'SR_B7': image.select('SR_B7')
    }).rename('TCB')
  bsi = image.expression(
      '(( X + Y ) - (A + B)) /(( X + Y ) + (A + B)) ', {
        'X': image.select('SR_B4'), #red
        'Y': image.select('SR_B6'),  #swi1
        'A': image.select('SR_B5'), # nir
        'B': image.select('SR_B2'), # blue
  }).rename('bsi')
  return image.addBands(ndvi).addBands(ndbi).addBands(ndwi).addBands(bsi)

june = addIndices(june)

# Function to Normalize Image
def normalize(image):
  bandNames = image.bandNames()
  # Compute min and max of the image
  minDict = image.reduceRegion(**{
    'reducer': ee.Reducer.min(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9,
    'bestEffort': True,
    'tileScale': 16
  })
  maxDict = image.reduceRegion(**{
    'reducer': ee.Reducer.max(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9,
    'bestEffort': True,
    'tileScale': 16
  })
  mins = ee.Image.constant(minDict.values(bandNames))
  maxs = ee.Image.constant(maxDict.values(bandNames))

  normalized = image.subtract(mins).divide(maxs.subtract(mins))
  return normalized

# Function to Standardize Image

def standardize(image):
  bandNames = image.bandNames()
  # Mean center the data to enable a faster covariance reducer
  # and an SD stretch of the principal components.
  meanDict = image.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9,
    'bestEffort': True,
    'tileScale': 16
  })
  means = ee.Image.constant(meanDict.values(bandNames))
  centered = image.subtract(means)

  stdDevDict = image.reduceRegion(**{
    'reducer': ee.Reducer.stdDev(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9,
    'bestEffort': True,
    'tileScale': 16
  })
  stddevs = ee.Image.constant(stdDevDict.values(bandNames))

  standardized = centered.divide(stddevs)

  return standardized

june= normalize(june)
june = standardize(june)


# validate with Baringo

#baringo = ee.Geometry.BBox(35.528285, 0.170590, 35.840163, -0.097081)

# Classify the image.
classified = june.classify(trainedClassifier)

#display the map
#visualize classification
Map.addLayer(classified.clip(baringo),
             {'min': 10, 'max': 206,
              'palette': [
                          'cadetblue',
                          'chartreuse',
                          'chocolate',
                          'coral',
                          'cornflowerblue',
                          'cornsilk',
                          'crimson',
                          'cyan',
                          'darkblue',
                          'darkcyan',
                          'darkgoldenrod',
                          'darkgray',
                          'darkgreen',
                          'darkkhaki',
                          'darkmagenta',
                          'darkolivegreen',
                          'darkorange',
                          'darkorchid',
                          'darkred',
                          'darksalmon',
                          'darkseagreen',
                          'darkslateblue',
                          'darkslategrey']},
             'classified')

Map

Map(bottom=524398.0, center=[0.0652313091502137, 35.684623718261726], controls=(WidgetControl(options=['positi…

In [ ]:
# define legend dictionary
legend_dict = {
    'Avocado': 'cadetblue',                  #2          
    'Bay Leaves':  'chartreuse',             #6        
    'Calliandra': 'chocolate',               #13        
    'Coriander': 'coral',                    #24        
    'Cauliflower': 'cornflowerblue',         #17        
    'Bananas': 'cornsilk',                   #3         
    'Courgette': 'crimson',                  #27        
    'Exotic Chinese Vegetables': 'cyan',     #33        
    'Broccoli': 'darkblue',                  #10        
    'Spinach': 'darkcyan',                   #103      
    'Thorned Melon': 'darkgoldenrod',	       #115      
    'Corn Maize': 'darkgray',	               #25       
    'Passion Fruit': 'darkgreen',	           #76       
    'Cabbage': 'darkkhaki',                  #12       
    'Pumpkin': 'darkmagenta',                #89       
    'Onions': 'darkolivegreen',	             #70       
    'Tree Tomato': 'darkorange',             #118      
    'Red Radish': 'darkorchid',              #93       
    'Potato': 'darkred',                     #87      
    'Sukuma Wiki': 'darksalmon',             #108     
    'Maize': 'darkseagreen',                 #57      
    'Napier Grass': 'darkslateblue',         #67      
    'Tea':'darkslategray',                    #113
    'Urban or Built up area': 'deeppink',             #200
    'Agriculture Land': 'sienna',                   #201
    'Forest Land':'lime',                  #203
    'Barren Land':'olive',                #206 
}
	
Map.add_legend(legend_dict=legend_dict)
Map